<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/SPACE_FP_OODA_LOOP_AI_AGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic -q
!pip install colab-env --quiet

import anthropic
import os
import colab_env
import json

In [2]:
api_key = os.environ["CLAUDE3_API_KEY"]
#model="claude-3-opus-20240229"
model="claude-3-5-sonnet-20240620"

client = anthropic.Anthropic(
    api_key=api_key,
)


message = client.messages.create(
    #model="claude-3-opus-20240229",
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content[0].text)

Hello! It's nice to meet you. How can I assist you today?


In [ ]:
#!cp -pr /root/.cache/kagglehub/datasets/ryleymcconkey/ml-turbulence-dataset/versions/6/REF.csv /content/gdrive/MyDrive/datasets/turbulence/REF.csv

In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("erevear/space-weather-solar-geomagnetic-indices")

#print("Path to dataset files:", path)

#!cp  /root/.cache/kagglehub/datasets/erevear/space-weather-solar-geomagnetic-indices/versions/2/daily_solar_data.csv  /content/gdrive/MyDrive/datasets/SW/

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
daily_solar_data_space_df = pd.read_csv('/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
daily_solar_data_space_df

,Date,Radio Flux 10.7cm,Sunspot Number,Sunspot Area (10^6 Hemis.),New Regions,Stanford Mean Solar Field (GOES15),Stanford Background X-Ray Flux,Flares: C,Flares: M,Flares: X,Flares: S,Flares: 1,Flares: 2,Flares: 3
0,1997-01-01,72,0,0,0,*,A0.5,0,0,0,-1,-1,-1,-1
1,1997-01-02,72,0,0,0,*,A0.5,0,0,0,-1,-1,-1,-1
2,1997-01-03,73,0,0,0,3,A0.5,0,0,0,-1,-1,-1,-1
3,1997-01-04,74,13,10,1,*,A0.6,0,0,0,-1,-1,-1,-1
4,1997-01-05,74,15,20,0,5,A0.7,0,0,0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10325,2024-04-08,125,79,470,1,-999,*,2,0,0,0,0,0,0
10326,2024-04-09,124,64,405,0,-999,*,2,0,0,0,0,0,0
10327,2024-04-10,131,54,410,1,-999,*,11,0,0,3,0,0,0
10328,2024-04-11,144,81,560,1,-999,*,10,1,0,4,0,0,0


In [5]:
space_weather_df = pd.read_csv('/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv')
space_weather_df['Date'] = pd.to_datetime(space_weather_df['Date'])

In [12]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import anthropic
import os
import re
import time
from tqdm import tqdm

# Get Claude API key from environment variable
claude_api_key = os.environ.get("CLAUDE3_API_KEY")
if not claude_api_key:
    raise ValueError(
        "CLAUDE3_API_KEY environment variable not set. Please set it before running the code."
    )

model = "claude-3-5-sonnet-20240620"
client = anthropic.Anthropic(api_key=claude_api_key)

space_weather="/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv"

# Load space weather data from the provided path
space_weather_df = pd.read_csv(space_weather)
space_weather_df["Date"] = pd.to_datetime(space_weather_df["Date"])


class SpaceEnvironment:
    def __init__(self, origin, destination, space_weather_df):
        self.origin = origin
        self.destination = destination
        self.space_weather_df = space_weather_df  # Store the DataFrame

    def get_possible_trajectories(self):
        # Define trajectories, including Lunar Gateway as a waypoint if needed
        trajectories = {
            ("Earth orbit", "Moon surface"): [
                {
                    "trajectory_id": "T1",
                    "delta_v": 3.9,
                    "duration": 3.5,
                    "fuel_required": 1200,
                    "waypoints": ["Lunar Gateway"],  # Include Lunar Gateway as a waypoint
                },
                # ... (add more trajectories) ...
            ]
        }
        return trajectories.get((self.origin, self.destination), [])

    def get_space_weather_data(self, location):
        # Extract data for the given location from the DataFrame
        latest_data = self.space_weather_df.iloc[-1]  # Get the latest data row
        # Adjust logic based on your data format and column names
        return {
            "solar_flare_risk": "High" if latest_data["Flares: C"] > 0 else "Low",  # Corrected column name
            "radiation_level": "Moderate",  # Replace with relevant logic based on other columns
        }


class SpaceFlightAgent:
    def __init__(self, environment):
        self.environment = environment
        self.space_weather_model = LogisticRegression()
        self.train_space_weather_model()

    def train_space_weather_model(self):
        # Feature engineering and model training
        space_weather_df["high_solar_activity"] = (
            (space_weather_df["Sunspot Number"] > 50)  # Corrected column name
            | (space_weather_df["Radio Flux 10.7cm"] > 150)  # Corrected column name
        ).astype(int)
        target = space_weather_df["high_solar_activity"]
        features = space_weather_df[["Sunspot Number", "Radio Flux 10.7cm"]]  # Corrected column names
        self.space_weather_model.fit(features, target)
        print("Space weather model trained successfully!")

    def generate_space_flight_plan(self):
        origin_weather = self.environment.get_space_weather_data(
            self.environment.origin
        )
        destination_weather = self.environment.get_space_weather_data(
            self.environment.destination
        )
        possible_trajectories = self.environment.get_possible_trajectories()

        print(f"Space weather at {self.environment.origin}: {origin_weather}")
        print(
            f"Space weather at {self.environment.destination}: {destination_weather}"
        )
        print("Available trajectories:", possible_trajectories)

        if not possible_trajectories:
            return "No trajectories found."

        # OODA Loop:
        print('\n')
        num_iterations = 5  # Define the number of iterations
        for _ in tqdm(range(num_iterations), desc="OODA Loop"):
            # Observe:
            # (In this example, we are using the existing space_weather_df)
            # For real-time updates, you would need to fetch new data here
            # and update the space_weather_df accordingly.

            # Orient:
            latest_data = self.environment.space_weather_df.iloc[-1]  # Latest data row
            current_sunspot_number = latest_data["Sunspot Number"]  # Corrected column name
            current_radio_flux = latest_data["Radio Flux 10.7cm"]  # Corrected column name

            origin_features = [[current_sunspot_number, current_radio_flux]]
            destination_features = [[current_sunspot_number, current_radio_flux]]

            # Create DataFrames with feature names for prediction
            origin_features_df = pd.DataFrame(origin_features, columns=["Sunspot Number", "Radio Flux 10.7cm"])
            destination_features_df = pd.DataFrame(destination_features, columns=["Sunspot Number", "Radio Flux 10.7cm"])

            origin_weather["solar_flare_risk"] = (
                "High"
                if self.space_weather_model.predict(origin_features_df)[0] == 1
                else "Low"
            )
            destination_weather["solar_flare_risk"] = (
                "High"
                if self.space_weather_model.predict(destination_features_df)[0] == 1
                else "Low"
            )

            # Decide:
            # ... (Add your decision logic here based on origin_weather,
            # destination_weather, etc.) ...
            chosen_trajectory = possible_trajectories[0]  # Example: Choose the first trajectory

            # Act:
            # Call Claude to generate/update the plan
            prompt = f"""
            Generate a detailed space flight plan for a mission from {self.environment.origin} to the {self.environment.destination},
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {chosen_trajectory}
            - Solar flare risk at origin: {origin_weather['solar_flare_risk']}
            - Solar flare risk at destination: {destination_weather['solar_flare_risk']}
            # ... (Add other instructions/details to the prompt) ...
            """
            response = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                max_tokens=2048,
                messages=[{"role": "user", "content": prompt}],
            )
            space_flight_plan = response.content[0].text

            # Post-processing
            space_flight_plan = re.sub(r"\n+", "\n", space_flight_plan)
            space_flight_plan = re.sub(r"## (.*)", r"\n\n**\1**:", space_flight_plan)

            print('\n')
            print("\nGenerated Space Flight Plan:")
            print(space_flight_plan)
            print('\n')

            # ... (Add a delay to control loop frequency) ...
            # time.sleep(60)  # Example: Wait 60 seconds (commented out or adjusted)


# Example usage
environment = SpaceEnvironment(
    origin="Earth orbit",
    destination="Moon surface",
    space_weather_df=space_weather_df,  # Pass the DataFrame
)

agent = SpaceFlightAgent(environment)
agent.generate_space_flight_plan()

Space weather model trained successfully!
Space weather at Earth orbit: {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}
Space weather at Moon surface: {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}
Available trajectories: [{'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}]




OODA Loop:  20%|██        | 1/5 [00:12<00:50, 12.52s/it]




Generated Space Flight Plan:
Based on the provided information and considerations, here's a detailed space flight plan for a mission from Earth orbit to the Moon surface, using NASA's Lunar Gateway as a staging point:
Mission Overview:
- Origin: Earth Orbit
- Destination: Moon Surface
- Staging Point: NASA's Lunar Gateway
- Chosen Trajectory: T1
- Delta-V Required: 3.9 km/s
- Mission Duration: 3.5 days
- Fuel Required: 1200 kg
Pre-Launch Preparations:
1. Conduct thorough systems checks on the spacecraft and all critical components.
2. Load necessary supplies, equipment, and scientific instruments for lunar surface operations.
3. Verify life support systems and radiation shielding are functioning optimally.
4. Brief crew on mission parameters, emergency procedures, and solar flare risks.
Launch and Earth Orbit Departure:
1. Launch from Earth to designated Earth orbit.
2. Perform system checks and final preparations for trans-lunar injection.
3. Execute trans-lunar injection burn to b

OODA Loop:  40%|████      | 2/5 [00:26<00:40, 13.50s/it]




Generated Space Flight Plan:
Based on the provided information and considerations, here's a detailed space flight plan for a mission from Earth orbit to the Moon surface, using NASA's Lunar Gateway as a staging point:
Mission Overview:
- Origin: Earth orbit
- Destination: Moon surface
- Staging Point: NASA's Lunar Gateway
- Chosen Trajectory: T1
- Delta-V required: 3.9 km/s
- Mission Duration: 3.5 days
- Fuel Required: 1200 kg
Pre-Launch Preparations:
1. Conduct thorough spacecraft systems checks and final crew briefings.
2. Monitor solar activity closely due to high solar flare risk at both origin and destination.
3. Ensure all radiation shielding systems are fully operational.
4. Load extra supplies and emergency equipment to account for potential delays due to solar activity.
Launch and Earth Departure:
1. Launch from Earth orbit using a powerful upper stage rocket.
2. Perform Trans-Lunar Injection (TLI) burn to begin trajectory T1 towards the Lunar Gateway.
3. Deploy solar panel

...




Generated Space Flight Plan:
Based on the provided information, here's a detailed space flight plan for a mission from Earth orbit to the Moon surface, utilizing NASA's Lunar Gateway as a staging point:
Mission Overview:
- Origin: Earth orbit
- Destination: Moon surface
- Staging Point: NASA's Lunar Gateway
- Chosen Trajectory: T1
  - Delta-V: 3.9 km/s
  - Duration: 3.5 days
  - Fuel Required: 1200 kg
  - Waypoints: Lunar Gateway
Pre-Launch Preparations:
1. Conduct thorough systems checks on the spacecraft, ensuring all components are functioning optimally.
2. Load necessary supplies, scientific equipment, and fuel (minimum 1200 kg for the chosen trajectory).
3. Brief crew on mission parameters, safety protocols, and contingency plans.
4. Monitor solar activity closely due to high solar flare risk at both origin and destination.
Launch and Earth Departure:
1. Launch from Earth orbit, initiating the T1 trajectory.
2. Perform necessary course corrections to align with the planned traj

OODA Loop:  80%|████████  | 4/5 [00:51<00:12, 12.82s/it]




Generated Space Flight Plan:
Based on the provided information, here's a detailed space flight plan for a mission from Earth orbit to the Moon surface, using NASA's Lunar Gateway as a staging point:
Mission Overview:
- Origin: Earth orbit
- Destination: Moon surface
- Staging Point: NASA's Lunar Gateway
- Chosen Trajectory: T1
- Delta-V requirement: 3.9 km/s
- Mission Duration: 3.5 days
- Fuel Required: 1200 kg
Pre-Launch Preparations:
1. Conduct thorough systems checks on the spacecraft and ensure all components are flight-ready.
2. Load necessary supplies, scientific equipment, and fuel.
3. Brief crew on mission parameters, safety protocols, and contingency plans.
4. Monitor space weather forecasts, particularly due to high solar flare risk at both origin and destination.
Launch and Earth Orbit Departure:
1. Launch from Earth to low Earth orbit (LEO) using a suitable launch vehicle.
2. Perform system checks in LEO and prepare for trans-lunar injection (TLI).
3. Execute TLI burn to

OODA Loop: 100%|██████████| 5/5 [01:06<00:00, 13.36s/it]




Generated Space Flight Plan:
Based on the provided information and considerations, here's a detailed space flight plan for a mission from Earth orbit to the Moon surface, using NASA's Lunar Gateway as a staging point:
Mission Overview:
- Origin: Earth Orbit
- Destination: Moon Surface
- Staging Point: NASA's Lunar Gateway
- Chosen Trajectory: T1
  - Delta-V: 3.9 km/s
  - Duration: 3.5 days
  - Fuel Required: 1200 kg
  - Waypoints: Lunar Gateway
Pre-Launch Preparations:
1. Conduct thorough spacecraft systems checks and ensure all redundancies are in place.
2. Load extra radiation shielding materials due to high solar flare risks at both origin and destination.
3. Brief crew on solar flare emergency procedures and ensure all protective equipment is readily accessible.
4. Update flight software with latest solar activity predictions and trajectory calculations.
Launch and Earth Orbit Operations:
1. Launch spacecraft from Earth to Low Earth Orbit (LEO).
2. Perform systems check in LEO a